In [ ]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_handoff_tool, create_swarm
from langgraph.store.memory import InMemoryStore
from langmem import create_manage_memory_tool, create_search_memory_tool, ReflectionExecutor, create_memory_store_manager


from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model='claude-3-opus-20240229')

# Set up storage 
store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small",
    }
) 

Logos = create_react_agent(
    model,
    [create_handoff_tool(agent_name="Ethos", description="Transfer to Ethos, who is good writing and expressing herself"),
     create_manage_memory_tool(namespace=("memories",)),
     create_search_memory_tool(namespace=("memories",))
    ],
    prompt="",
    name="Logos",
    store=store,
)

Ethos = create_react_agent(
    model,
    [create_handoff_tool(agent_name="Logos", description="Transfer to Logos, who is good at planning and analysis"),
     create_manage_memory_tool(namespace=("memories",)),
     create_search_memory_tool(namespace=("memories",))
    ],
    prompt="You are Ethos",
    name="Ethos",
    store=store,
)


checkpointer = InMemorySaver()
workflow = create_swarm(
    [Logos, Ethos],
    default_active_agent="Logos"
)

app = workflow.compile(checkpointer=checkpointer)

ValueError: Default active agent 'Alice' not found in routes ['Logos', 'Ethos']

In [ ]:

config = {"configurable": {"thread_id": "1"}}
turn_1 = app.invoke(
    {"messages": [{"role": "user", "content": "i'd like to speak to Bob"}]},
    config,
)
print(turn_1)
turn_2 = app.invoke(
    {"messages": [{"role": "user", "content": "what's 5 + 7?"}]},
    config,
)
print(turn_2)